In [53]:
%whos str

Variable     Type    Data/Info
------------------------------
global_url   str     https://realty.yandex.ru/<...>/kvartira/odnokomnatnaya/
html         str     <!doctype html><html lang<...>\n</script></body></html>


In [31]:
x = 5
y = [1,3,4,5,6]

In [2]:
def get_vars():
    np.math.acos(2)
    csv.DictWriter(html)
    sleep(23)
    requests.get(url)
    

!pip3 install https://github.com/ipython-contrib/jupyter_contrib_nbextensions/tarball/master
!pip3 install jupyter_nbextensions_configurator
!jupyter contrib nbextension install --user

In [1]:
import requests
from bs4 import BeautifulSoup
import re
from time import sleep
import csv
import numpy as np

# Парсинг https://wordpress.org/plugins/

In [14]:
def get_html(url):
    r = requests.get(url)
    return r.text

url = 'https://realty.yandex.ru/moskva/snyat/kvartira/odnokomnatnaya/'
html = get_html(url)
soup = BeautifulSoup(html, 'lxml')

In [15]:
len(soup.find_all('section'))

def get_featured_plugins(soup, name='Featured Plugins'):
    for section in soup.find_all("section", class_="plugin-section"):
        temp = section.find('h2').text
        if temp == name:
            fp_=section
    try:
        return fp_
    except UnboundLocalError as err:
        print('Такого раздела нет, ребята!')
        raise err

#fp = get_featured_plugins(soup, name='Featured Plugins')

def get_data(fp):
    sites = []
    names = []
    ratings = []
    fp1 = fp.find_all('h3', class_="entry-title")
    fp2 = fp.find_all('div', class_="plugin-rating")
    for i in fp1:
        sites.append(i.a['href'])
        names.append(i.a.string)
    for i in fp2:
        ratings.append(i.text)
    return (sites, names, ratings)



#data = get_data(fp)


import csv

# with open('./file.csv', "w", newline="\n") as file:
#     writer = csv.writer(file)
#     writer.writerow(data)

#### Способ построения дерева разметки и запроса для поиска без обзора визуального

Сначала идентифицируем ту единицу поиска, которая одна из многих. Визуально, потом построения запроса как получится:

In [7]:
child = soup.find_all("section", class_="plugin-section")[1].find('a', href="https://wordpress.org/plugins/classic-editor/")

IndexError: list index out of range

Потом выводим полный путь родителей - путь наверх:

In [7]:
for parent in child.parents:
    print(parent.name)

NameError: name 'child' is not defined

То же самое проделаем для второй единицы поиска:

In [136]:
child1 = soup.find_all("section", class_="plugin-section")[2].find('a', href="https://wordpress.org/plugins/gutenberg/")

In [137]:
for parent in child1.parents:
    print(parent.name)

div
article
section
main
div
div
body
html
[document]


Мы видим, что пути пересекаются на уровне section. Еще более важная информация заключается в том, что после section нижеследующий тег для поиска - article.
Попробуем выделить нашу единицу поиска уже путем подстановки article и уже визуального анализа разметки, что теперь легко сделать ввиду размерности разметки:

In [138]:
soup.find_all("section", class_="plugin-section")[2].find_all('article')[0].find('a')['href']

'https://wordpress.org/plugins/gutenberg/'

In [38]:
soup.find_all("section", class_="plugin-section")[2].find_all('article')[0].h3.find('a')['href']

'https://wordpress.org/plugins/gutenberg/'

Вот и все. Теперь задача поиска сайтов плагинов сводится к написанию итератора for i in ...

In [31]:
sections = soup.find_all("section", class_="plugin-section")

In [144]:
spisok = []
for section in sections:
    articles = section.find_all('article')
    for article in articles:
        #temp = article.find('a')['href']
        temp = article.h3.find('a')['href']
        if article.h3.find('a').text!=None:
            spisok.append(temp)

# Парсинг https://coinmarketcap.com/

In [8]:
def get_html(url):
    r = requests.get(url)
    return r.text

url = 'https://coinmarketcap.com/'
html = get_html(url)
soup = BeautifulSoup(html, 'lxml')

In [9]:
trows = soup.tbody.find_all('tr')

In [10]:
trows[0].a['href']

'/currencies/bitcoin/'

In [11]:
trows[0].a.text

'Bitcoin1BTC'

In [12]:
data = []
for row in trows:
    href = row.a['href']
    name = row.a.text
    data.append((href, name))

#### Хотя можно было так, намного эффективнее, перебор по тегам td:

In [148]:
trows = soup.tbody.find_all('tr')

In [199]:
tds = trows[17].find_all('td')
tds

[<td><span></span></td>,
 <td><span></span></td>,
 <td><a class="cmc-link" href="/currencies/solana/"><span class="circle"></span><span>Solana</span><span class="crypto-symbol">SOL</span></a></td>,
 <td><span>$<!-- -->26.78</span></td>,
 <td><span></span></td>]

In [200]:
tds[2].a['href']

'/currencies/solana/'

In [201]:
tds[3].find('span').text

'$26.78'

#### РЕГУЛЯРНОЕ ВЫРАЖЕНИЕ ПРИМЕНЕНО!!! СМ:

In [130]:
trows[4].find_all('div', class_=re.compile("price"))

[<div class="price___3rj7O"><a class="cmc-link" href="/currencies/cardano/markets/">$1.39</a></div>]

# Парсинг рубрикатора Яндекса. 2990 объявлений показывает
написать функцию которая удаляет \xa0 и заменяет пробелом, на уровне переменной html

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import random
import csv
global_url = 'https://realty.yandex.ru/moskva/snyat/kvartira/odnokomnatnaya/'

### МОДИФИЦИРУЕМ ОБРАЩЕНИЕ К САЙТУ, НА СЛУЧАЙ ЕСЛИ ЗАБАНЯТ

def get_html(url):
    r = requests.get(url)
    if r.ok:
        return r.text
    else:
        print(r.status_code)
        
###    

def get_soup(html):
    html1=html.encode('l1').decode()
    soup = BeautifulSoup(html1, 'lxml')
    return soup

def write_csv(data):
    with open('arenda.csv', 'a') as f:
        writer = csv.writer(f)

        writer.writerow([data['space'],
                         data['metro'],
                         data['price'],
                         data['description']])

In [2]:
def get_data_from_page(url):
    html = get_html(url)
    soup = get_soup(html)
    page_offers = soup.find_all('li', class_='OffersSerpItem OffersSerpItem_view_desktop OffersSerpItem_format_full OffersSerp__list-item OffersSerp__list-item_type_offer')
    for offer in offers:
        space = offer.find('h3', class_='OffersSerpItem__title').text
        metro = offer.find('span', class_='MetroStation').text
        price = offer.find('span', class_='price').text
        description = offer.find('p', class_='OffersSerpItem__description').text
        data = {'space': space,
               'metro': metro,
               'price': price,
               'description': description}
        #print(data)
        write_csv(data)

In [10]:
def main(global_url):
    pages = range(30,100)
    for page in pages:
        url = global_url + '?page=' + str(page)
        time.sleep(random.randint(1,10))
        get_data_from_page(url)

###### НАДО СОЗДАТЬ СНАЧАЛА ФАЙЛ:
with open('arenda.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(['space', 'metro', 'price', 'description'])

###### ПОТОМ ЗАПУСК:

In [11]:
main(global_url)

###### МОЖНО ОРГАНИЗОВАТЬ ЦИКЛ  WHILE, ПОКА ЕСТЬ КНОПКА "следующая":  (в конце break можно использовать)

In [4]:
html = get_html(global_url)
soup = get_soup(html)

In [6]:
soup.find(string=re.compile('Следующая')).parent

<a class="Pager__radio-link" href="/moskva/snyat/kvartira/odnokomnatnaya/?page=1">Следующая<i class="Icon Icon_type_arrow Icon Icon_direction_right Icon_animate-direction Pager__next-arrow"><svg viewbox="0 0 9 5" xmlns="http://www.w3.org/2000/svg"><path class="shape" d="M4.5 3.7L.7 0 0 .7 4.5 5 9 .7 8.3 0z"></path></svg></i></a>

In [5]:
soup.find_all('a', class_='Pager__radio-link')[-1].text=='Следующая'

True

In [43]:
while True:
    if (random.randint(1,100000) + random.randint(1,100000)) == 5000:
        break

###### ВОТ ЧТО ДАВНО ИСКАЛИ. ЕСЛИ ЕСТЬ ФРАГМЕНТ ТЕКСТА НА СТРАНИЦЕ, ЛЕГКО УЗНАТЬ К КАКОМУ ТЕГУ ОН ПРИНАДЛЕЖИТ, ДА И ВООБЩЕ ВСЕХ PARENTS НАЙТИ:

In [59]:
soup.find(string=re.compile('Следующая')).parent

<a class="Pager__radio-link" href="/moskva/snyat/kvartira/odnokomnatnaya/?page=21">Следующая<i class="Icon Icon_type_arrow Icon Icon_direction_right Icon_animate-direction Pager__next-arrow"><svg viewbox="0 0 9 5" xmlns="http://www.w3.org/2000/svg"><path class="shape" d="M4.5 3.7L.7 0 0 .7 4.5 5 9 .7 8.3 0z"></path></svg></i></a>

In [60]:
soup.find(string=re.compile('Следующая')).parent.parent

<span class="Button__text"><a class="Pager__radio-link" href="/moskva/snyat/kvartira/odnokomnatnaya/?page=21">Следующая<i class="Icon Icon_type_arrow Icon Icon_direction_right Icon_animate-direction Pager__next-arrow"><svg viewbox="0 0 9 5" xmlns="http://www.w3.org/2000/svg"><path class="shape" d="M4.5 3.7L.7 0 0 .7 4.5 5 9 .7 8.3 0z"></path></svg></i></a></span>

In [61]:
soup.find(string=re.compile('Следующая')).parent.parent.parent

<button class="Button Button_js_inited Button_size_m Button_theme_islands Button_type_button" type="button"><span class="Button__text"><a class="Pager__radio-link" href="/moskva/snyat/kvartira/odnokomnatnaya/?page=21">Следующая<i class="Icon Icon_type_arrow Icon Icon_direction_right Icon_animate-direction Pager__next-arrow"><svg viewbox="0 0 9 5" xmlns="http://www.w3.org/2000/svg"><path class="shape" d="M4.5 3.7L.7 0 0 .7 4.5 5 9 .7 8.3 0z"></path></svg></i></a></span></button>

###### ЯРКИЙ ПРИМЕР. МЫ ВИДИМ НА СТРАНИЦЕ, ЧТО ОДНА ИЗ КВАРТИР НАХОДИТСЯ В МЕДВЕДКОВО. МЫ ДОБАВЛЯЕМ ПОСТЕПЕННО .PARENT.TEXT И СМОТРИМ, ЧТОБЫ НАЧАЛИ ОТПЕЧАТЫВАТЬСЯ ВСЕ КВАРТИРЫ. ПОТОМ .TEXT УБИРАЕМ И ПИШЕМ СЛЕДУЮЩИЙ КОД, КОГДА НАШЛИ САМЫЙ РОДИТЕЛЬСКИЙ ДЛЯ ВСЕХ КВАРТИР PARENT:

In [78]:
flats = soup.find(string=re.compile('Медведково')).parent.parent.parent.parent.parent.parent.parent.parent.parent.parent.parent

In [ ]:
for flat in flats:
    print(flat.text)

+ 1237,8 м², 1-комнатная11 этаж из 14проезд Шокальского, 35Медведково 13 мин.31 000 ₽ / мес.от Яндекс.АрендыПроверено в Росреестре?Квартира сдаётся с Яндекс.Арендой, а значит все для вас: - без залога - без единоразовой комиссии - с поддержкой от наших специалистов в процессе проживания Мы можем показать вам квартиру онлайн - благодаря уникальному 3Д туру - это также детально,Показать телефонВ избранноеЯндекс.Аренда, сервис Яндекса6 минут назад
+ 440 м², 1-комнатная3 этаж из 12Ботанический переулок, 5Проспект Мира 8 мин.120 000 ₽ / мес.?ID 32519 Апартаменты в стиле модерн в отеле "Гарден" на охраняемой территории Ботанического сада. Апартаменты созданы для длительного проживания с максимальным комфортом и станут идеальным местом для проживания в долгосрочных командировках иПоказать телефонВ избранноеSavills, агентство5 часов назад
+ 1520 м², 1-комнатнаяЖК «Атмосфера», 1 этаж из 14Совхозная улица, 8БЛюблино 10 мин.30 000 ₽ / мес.?Отличная студия в ЛюблиноПоказать телефонВ избранноеАН "SANDI", агентство4 часа назад
+ и так далее...

#### РЕГУЛЯРНЫЕ ВЫРАЖЕНИЯ. PYTHEX.ORG

\d{1,9} - любое количество только цифр. диапазон цифр - от 1 до 9

In [ ]:
def get_salary(s):
    # salary: 2700 usd per month
    pattern = r'\d{1,9}'
    # salary = re.findall(pattern, s)[0]
    print(salary)

    # СПРАВОЧНИК НАИБОЛЕЕ ЧАСТО ПРИМЕНЯЕМЫХ СИМВОЛОВ В РЕГУЛЯРНЫХ ВЫРАЖЕНИЯХ:
    # ^ - начало строки
    # $ - конец строки
    # . - любой символ
    # + - неограниченное количество вхождений
    # '\d' - цифра
    # '\w' - буквы, цифры, _
    # {1,9} - количество вхождений от 1 до 9

ПРИМЕР:   '^@\w+'   - слово начинается на @ (аккаунты допустим), потом идут любые буквы, цифры в неограниченном количестве.
Или ниже - слова на букву М, число букв/цифр потом от 6 до 12:

In [84]:
soup.find_all(string=re.compile('^М\w{6,12}'))

['Монолитный',
 'Медведково',
 'Москворечье',
 'Московская область',
 'Марксистская',
 'Марьина роща',
 'Марьино',
 'Маяковская',
 'Медведково',
 'Международная',
 'Менделеевская',
 'Минская',
 'Мичуринский проспект',
 'Мнёвники',
 'Молодёжная',
 'Москворечье',
 'Мякинино',
 'Марушкинское',
 'Марфино',
 'Марьина Роща',
 'Марьино',
 'Матушкино',
 'Метрогородок',
 'Мещанский',
 'Михайлово-Ярцевское',
 'Можайский',
 'Молжаниновский',
 'Москворечье-Сабурово',
 'Московский',
 'Мосрентген',
 'Московская область',
 'Магаданская область',
 'Мордовия',
 'Мурманская область',
 'Мобильная версия']

# ЗАПИСЬ С ПАРСИНГА В БАЗУ ДАННЫХ POSTGRESQL

Первоначальные настройки POSTGRESQL для начала работы в PGAdmin4.
Первый раз будем логиниться от имени создаваемого системного юзера postgres, при настройках по-умолчанию для него используется авторизация типа peer, никакого пароля постгре не запросит:
 * $ sudo -u postgres psql

Далее уже в оболочке постгреса (куда вы должны попасть после предыдущей команды) выполним смену пароля пользователя:
 * $ \password postgres
-- введите новый пароль (скорее всего дважды). У меня будет цифра 1.

Создание базы данных (название test): # если не работает, вводить руками, не копировать #
 * $ CREATE DATABASE test;

Просмотр всех имеющихся баз данных:
 * $ \l

Выходим из постгре:
 * $ \q
 
Или выходим из оболочки постгре командой:
 * $ exit

Пакеты, в том числе ORM, необходимые для работы кода ниже:
 * $  pip install peewee psycopg2 psycopg2-binary

Чтобы сделать дамп (файл базы данных для передачи заинтересованным лицам):
 * $ pg_dump -U postgres -h localhost test > coins.sql

In [ ]:
import csv
from peewee import *


#file = open('.cmc.csv')
FILENAME = "./help_files/cmc.csv"

### ВАЖНО: db как шаблон ПОДСОЕДИНЕНИЯ К УЖЕ СУЩЕСТВУЮЩЕЙ БАЗЕ test
db = PostgresqlDatabase(database='test', user='postgres',
password='1', host='localhost')

class Coin(Model):
    name = CharField()
    url = TextField()
    price = CharField()

    class Meta:
        database = db

def main():
    db.connect()
    db.create_tables([Coin])
    with open(FILENAME, "r") as f:
        order = ['name', 'url', 'price']
        reader = csv.DictReader(f, fieldnames=order)
        coins = list(reader)
        # 1 способ
        # for row in coins:
        #     coin = Coin(name=row['name'],
        #     url=row['url'], price=row['price'])
        #     coin.save()

        # 2 способ
        # with db.atomic():
        #     for row in coins:
        #         Coin.create(**row)

        # 3 способ, самый быстрый, пакетами запись
        with db.atomic():
            for index in range(0, len(coins), 100):
                Coin.insert_many(coins[index:index+100]).execute()

In [ ]:
if __name__=='__main__':
    main()

# 08 Парсинг данных подгружаемых через AJAX часть 1

Если при пагинации сайта, например https://www.liveinternet.ru/rating/ru/, смена страницы не влечет изменение адресной строки (так работает технология AJAX, когда JS отправляет на сервер запрос только текста для новой страницы для перехода, не перезагружает страницу целиком), заходим в инспектор кода страницы, потом:
 * вкладка Network страницы;
 * вкладка XHR
 * на самом сайте в пагинации страницы нажимаем номер следующей страницы
 * и в поле Name появляется другая ссылка, на которую собственно сайт и перешел. Адресная строка прежняя
 * видим в поле Response ответ сервера. Копируем эту настоящую ссылку из поля name и уже дальше ее и анализируем
 * в коде ниже мы работает с этой ссылкой, она возвращает просто текст, не объект Soup
 * поэтому работаем уже как с текстом - очищаем от пробелов, разбиваем по переносу строки и т.п.
 * по количеству страниц на сайте делаем цикл

In [1]:
import requests
import csv


def get_html(url):
    r = requests.get(url)
    return r.text


def write_csv(data):
    with open('websites.csv', 'a') as f:
        order = ['name', 'url', 'description', 'traffic', 'percent']
        writer = csv.DictWriter(f, fieldnames=order)
        writer.writerow(data)


def main():

    for i in range(0, 20): # здесь было 6428 страниц, убрал. говорят цикл до конца работает
        url = 'https://www.liveinternet.ru/rating/ru//today.tsv?page={}'.format(str(i))
        response = get_html(url)
        data = response.strip().split('\n')[1:]

        for row in data:
            columns = row.strip().split('\t')
            name = columns[0]
            url = columns[1]
            description = columns[2]
            traffic = columns[3]
            percent = columns[4]

            data = {'name': name,
                    'url': url,
                    'description': description,
                    'traffic': traffic,
                    'percent': percent}
            write_csv(data)






if __name__ == '__main__':
    main()

#### Примечание. Например, по адресу https://www.liveinternet.ru/stat/AIF/ (статистика для сайта Аиф, переход с рейтинга по ссылке) видно кучу информации для сайта Аиф:
(В отчете представлены общие показатели посещаемости сайта за сутки. Можно видеть за неделю, месяц и т.д.)

Можно подумать, как спарсить дополнительно все это, это же будет огромная база данных, возможно применить машинное обучение и какие-то интересные выводы по Рунету... Идеи:
* Прогноз рейтинга сайта через неделю, месяц и т.д. В зависимости от этих вышеперечисленных фич по дням/неделям

Там наверху есть отбор по категориям сайтов - спорт, обучение и т.п. Подумать надо тематическим отбором

# 09 Парсинг данных в несколько процессов

In [ ]:
import requests
import csv
from multiprocessing import Pool
from time import sleep

def get_html(url):
    # sleep(1)
    r = requests.get(url)
    return r.text


def write_csv(data):
    with open('websites.csv', 'a') as file:
        order = ['name', 'url', 'description', 'traffic', 'percent']
        writer = csv.DictWriter(file, fieldnames=order)
        writer.writerow(data)


def get_page_data(text):
        data = text.strip().split('\n')[1:]

        for row in data:
            columns = row.strip().split('\t')
            name = columns[0]
            url = columns[1]
            description = columns[2]
            traffic = columns[3]
            percent = columns[4]

            data = {'name': name,
                    'url': url,
                    'description': description,
                    'traffic': traffic,
                    'percent': percent}
            write_csv(data)


def make_all(url):
    text = get_html(url)
    get_page_data(text)


def main():
    #6320

    # ИЗЯЩНО ПОЛУЧИЛИ СПИСОК ВЕБ-СТРАНИЦ
    url = 'https://www.liveinternet.ru/rating/ru//today.tsv?page={}'
    urls = [url.format(str(i)) for i in range(1, 6321)]

    with Pool(20) as p:
        p.map(make_all, urls)




if __name__ == '__main__':
    main()

Мультипроцессинг хоть и должен был по идее увеличить скорость парсинга в 20 раз (with Pool(20)), такого быстродействия не наблюдалось. Скорее всего сервер ограничивает с одного IP такой траффик, по словам автора кода

# 10 Парсинг данных, подгружаемых с помощью JQuery

Если страница при прокручивании вниз не заканчивается, а продолжает подгружаться, то это работа JQuery

Как парсить? По аналогии с Ajax - смотрим при инспекте страницы не elements, а вкладку Network и там уже XHR

Ан нет. Requests при обращении к этим скрытым страницам возвращает код 403 - доступ запрещен...
Нам не хватает user agent (нашего браузера), который указывается в request headers.

Точнее так. В requests много параметров которые можно регулировать. Обычно начинают с подбора user agent, а дальше как пойдет... надо смотреть что там еще в в request headers.

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

def get_html(url):
    print(url)
    user_agent = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    r = requests.get(url, headers=user_agent)
    return r.text


def write_csv(data):
    with open('testimonials.csv', 'a') as f:
        order = ['author', 'since']
        writer = csv.DictWriter(f, fieldnames=order)
        writer.writerow(data)


def get_articles(html):
    soup = BeautifulSoup(html, 'lxml')
    ts = soup.find('div', class_='testimonial-container').find_all('article')
    return ts # [] or [a,b,s] - РЕАЛИЗОВАНО ХОРОШО. ЕСЛИ СПИСОК ПУСТОЙ, ЗНАЧИТ СТАТЕЙ НА ТЕКУЩЕЙ СТРАНИЦЕ НЕТ.
                                # ТОГДА ЭТО СЫГРАЕТ РОЛЬ В ЦИКЛЕ WHILE: BREAK ЦИКЛ ОСТАНОВИТ

def get_page_data(ts):
    for t in ts:
        try:
            since = t.find('p', class_='traxer-since').text.strip()
        except:
            since = ''
        try:
            author = t.find('p', class_='testimonial-author').text.strip()
        except:
            author = ''
        data = {'author': author, 'since': since}
        write_csv(data)


def main():
    # 1. Получение контейнера с отзывами и списка отзывов
    # 2. Если список есть, то парсим отзывы
    # 3. Если список пустой - цикл прерывается
    page = 1
    while True:
        #page = 1 ЗДЕСЬ АВТОР ОШИБСЯ. Я ПЕРЕНЕС ВЫРАЖЕНИЕ УРОВНЕМ ВЫШЕ. ИНАЧЕ ЦИКЛ БЕСКОНЕЧНО БЕГАЕТ ПО page = 1
        url = 'https://catertrax.com/why-catertrax/traxers/page/{}/'.format(str(page))

        articles = get_articles(get_html(url)) # [] or [1,2,3]

        if articles:
            get_page_data(articles)
            page = page + 1
        else:
            break


if __name__ == '__main__':
    main()

# 11 Парсинг ютюба

Важное замечание. Requests не обрабатывает Javascript. **Здесь идет мощная обработка ответов requests**

Здесь примечательно, что иницируется нажатие кнопки **"ЕЩЕ"**. Возвращается JSON-объект, и в теге/классе ответа содержится новая ссылка с видео

Если не получается обрабатывать такие случаи вытягивающихся страниц, можно спросить других спецов. Нанять один раз, и все.
 * Пишут на сайтах что нужен Selenium для обработки JS.
  * https://qna.habr.com/q/236162 Нужен пишут Selenium, PhantomJS
 * https://newtechaudit.ru/veb-skraping-request-html/  - requests_html умеет парсить JS скрипты

Ниже код автора, у него свой случай, код уже не работает, в ютюбе нет кнопки еще

In [47]:
import requests
from bs4 import BeautifulSoup
import csv


def get_html(url):
    r = requests.get(url)
    return r

def write_csv(data):
    with open('videos.csv', 'a') as f:
        order = ['name', 'url']
        writer = csv.DictWriter(f, fieldnames=order)
        writer.writerow(data)


def get_page_data(response):
    if 'html' in response.headers['Content-Type']:
        html = response.text
    else:
        html = response.json()['content_html']

    soup = BeautifulSoup(html, 'lxml')

    items = soup.find_all('h3', class_='yt-lockup-title')

    for item in items:
        name = item.text.strip()
        url = item.find('a').get('href')

        data = {'name': name, 'url': url}
        write_csv(data)


def get_next(response):
    if 'html' in response.headers['Content-Type']:
        html = response.text
    else:
        html = response.json()['load_more_widget_html']

    soup = BeautifulSoup(html, 'lxml')

    try:
        url = 'https://youtube.com' + soup.find('button', class_='load-more-button').get('data-uix-load-more-href')
    except:
        url = ''

    return url


def main():
    # url = 'https://www.youtube.com/browse_ajax?action_continuation=1&direct_render=1&continuation=4qmFsgJAEhhVQ09tNF9BbkxQTEVCVnJiby0tTjdrUEEaJEVnWjJhV1JsYjNNZ0FEZ0JZQUZxQUhvQk1yZ0JBQSUzRCUzRA%253D%253D'
    url = 'https://www.youtube.com/user/coolpropaganda/videos'
    # get_page_data(get_html(url))

    while True:
        response = get_html(url)
        get_page_data(response)

        url = get_next(response)

        if url:
            continue
        else:
            break






if __name__ == '__main__':
    main()

Поэкспериментирую с requests-html, он помогает вытащить все ссылки со страницы

In [72]:
from requests_html import HTMLSession  
session = HTMLSession()  
r = session.get('https://www.exler.ru/')

In [73]:
list(r.html.absolute_links)[-10:] # сократил выдачу, там много

['https://www.exler.ru/blog/vot-tebe-i-rossiyskiye-turisty.htm#comments',
 'https://www.exler.ru/ezhe/gorod-terrasa-dva-interesnejshih-osobnyaka-i-zamok-s-muzeem.htm#comments',
 'https://www.exler.ru/books/Exler%20-%20EPUB.zip',
 'https://www.exler.ru/blog/',
 'https://www.exler.ru/dieta/',
 'https://www.exler.ru/expromt/obzor-smartfona-xiaomi-mi-9-ubijca-li-on-flagmanov.htm',
 'https://www.exler.ru/info/firsttime.htm',
 'https://www.exler.ru/aliexpress/delimsya-nahodkami-iz-kitajskih-magazinov-39.htm',
 'https://www.exler.ru/ezhe/studiya-dzhejmsa-kemerona-ili-kak-sozdavalsya-avatar.htm#comments',
 'https://www.exler.ru/blog/yeshche-odno-nezavisimoye-smi-v-rossii-zakryvayetsya.htm#comments']

# 12 Используем прокси

In [16]:
import requests
from bs4 import BeautifulSoup
from random import choice
import csv

In [8]:
def get_proxy():
    html = requests.get('https://free-proxy-list.net/').text
    soup = BeautifulSoup(html, 'lxml')

    trs = soup.find('table', id='proxylisttable').find_all('tr')[1:20]
    proxies = []

    for tr in trs:
        tds = tr.find_all('td')
        ip = tds[0].text.strip()
        port = tds[1].text.strip()
        schema = 'https' if 'yes' in tds[6].text.strip() else 'http' # влияет от ресурса, который будем парсить
        proxy = {'schema': schema, 'address': ip + ':' + port}
        proxies.append(proxy)

    # return choice(proxies) ВАРИАНТ АВТОРА. ПОМЕНЯЛ, Т.К. МОЖНО ПРОСТО ЗАПИСАТЬ ВЕСЬ ПЕРЕЧЕНЬ, НЕ ОБНОВЛЯТЬ
    return proxies

Разовая операция парсинга и записи в файл данных прокси-серверов:

In [12]:
proxies = get_proxy()

In [20]:
with open('proxies_file.csv', 'a') as f:
    order = ['schema', 'address']
    writer = csv.DictWriter(f, fieldnames=order)
    for proxy in proxies:
        writer.writerow(proxy)

Считали данные из файла обратно:

In [22]:
with open('proxies_file.csv', "r") as f:
    order = ['schema', 'address']
    reader = csv.DictReader(f, fieldnames=order)
    proxies2 = list(reader)

In [27]:
choice(proxies2)

{'schema': 'http', 'address': '45.33.104.134:80'}

Проверяем на ресурсе, который показывает IP компьютера. Каждый раз он будет у нас разный:

In [65]:
def get_html(url, proxies2):
    # proxies = {'https': 'ipaddress:5000'}
    # p = get_proxy() # {'schema': '', 'address': ''}
    p = choice(proxies2)
    proxy = { p['schema']: p['address']  }
    r = requests.get(url, proxies=proxy, timeout=10)
    return r
    #return r.json()['origin']

In [69]:
def main():
    url = 'http://lenta.ru'
    print(get_html(url, proxies2))

if __name__ == '__main__':
    main()

<Response [503]>


In [67]:
p = choice(proxies2)
proxy = { p['schema']: p['address']  }
proxy

{'http': '5.252.161.48:8080'}